In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

import torch
from ipywidgets import interact

# from qsr_learning.data.data import draw, generate_objects

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Test the `Entity` Class

In [ ]:
import math

from PIL import Image
from qsr_learning.entity import Entity

In [ ]:
canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
entity1 = Entity(
    name="octopus",
    frame_of_reference="absolute",
    p=(30, 30),
    theta=0 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity1.draw(canvas, add_bbox=True, add_front=True)
entity2 = Entity(
    name="trophy",
    frame_of_reference="absolute",
    p=(60, 60),
    theta=90 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity2.draw(canvas, add_bbox=True, add_front=True)

# Test the Relations

In [ ]:
from ipywidgets import interact
from qsr_learning.binary_relation import above, below, left_of, right_of


@interact(
    frame_of_reference=(0, 1),
    x1=(0, 150),
    y1=(0, 150),
    theta1=(0, 360),
    x2=(0, 150),
    y2=(0, 150),
    theta2=(0, 360),
)
def test_spatial_relations(
    frame_of_reference=0, x1=32, y1=32, theta1=0, x2=50, y2=50, theta2=150
):
    canvas = Image.new("RGBA", (128, 128), (0, 0, 0, 255))
    entity1 = Entity(
        name="dog face",
        frame_of_reference={0: "absolute", 1: "intrinsic"}[frame_of_reference],
        p=(x1, y1),
        theta=theta1 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity1.draw(canvas, add_bbox=False, add_front=False)
    entity2 = Entity(
        name="cat face",
        frame_of_reference={0: "absolute", 1: "intrinsic"}[frame_of_reference],
        p=(x2, y2),
        theta=theta2 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity2.draw(canvas, add_bbox=False, add_front=False)
    display(canvas)

    for relation in [left_of, right_of, above, below]:
        if relation(entity1, entity2):
            print(entity1.name, relation.__name__, entity2.name)
#     for relation in [left_of, right_of, above, below]:
#         if relation(entity2, entity1):
#             print(entity2.name, relation.__name__, entity1.name)

In [ ]:
from ipywidgets import interact
from qsr_learning.ternary_relation import in_front_of, behind, left_of, right_of

@interact(
    x1=(0, 190),
    y1=(0, 190),
    theta1=(0, 360),
    x2=(0, 150),
    y2=(0, 150),
    theta2=(0, 360),
    x3=(0, 150),
    y3=(0, 150),
    theta3=(0, 360),
)
def test_spatial_relations(
    x1=63, y1=14, theta1=0, x2=40, y2=137, theta2=0, x3=112, y3=105, theta3=308
):
    canvas = Image.new("RGBA", (256, 256), (0, 0, 0, 0))

    entity1 = Entity(
        name="dog face",
        frame_of_reference="relative",
        p=(x1, y1),
        theta=theta1 / 360 * 2 * math.pi,
        size=(48, 48),
    )
    entity1.draw(canvas, add_bbox=False, add_front=False)

    entity2 = Entity(
        name="rabbit face",
        frame_of_reference="relative",
        p=(x2, y2),
        theta=theta2 / 360 * 2 * math.pi,
        size=(48, 48),
    )
    entity2.draw(canvas, add_bbox=False, add_front=False)

    entity3 = Entity(
        name="cat face",
        frame_of_reference="absolute",
        p=(x3, y3),
        theta=theta3 / 360 * 2 * math.pi,
        size=(48, 48),
    )
    entity3.draw(canvas, add_bbox=False, add_front=False, bbox_outline="black")

    display(canvas)
    if left_of(entity1, entity2, entity3):
        print(entity1.name, "left_of", entity2.name, "as_seen_from", entity3.name)
    if right_of(entity1, entity2, entity3):
        print(entity1.name, "right_of", entity2.name, "as_seen_from", entity3.name)
    if in_front_of(entity1, entity2, entity3):
        print(entity1.name, "in_front_of", entity2.name, "as_seen_from", entity3.name)
    if behind(entity1, entity2, entity3):
        print(entity1.name, "behind", entity2.name, "as_seen_from", entity3.name)

# Test the Samples

In [ ]:
from qsr_learning.data import draw_entities, generate_entities
from qsr_learning.entity import emoji_names

entities = generate_entities(
    entity_names=emoji_names[:5],
    frame_of_reference="absolute",
    w_range=(32, 32),
    h_range=(32, 32),
)
image = draw_entities(entities, add_bbox=True, add_front=False)
display(image)

# Test the Dataset

In [ ]:
import math

import torch
from ipywidgets import interact
from PIL import Image
from torch.utils.data import random_split

from qsr_learning.data import DRLDataset
from qsr_learning.entity import emoji_names

entity_names = ["octopus", "lion", "cat face", "dog face"]
relation_names = ["left_of", "right_of"]
dataset = DRLDataset(
    vocab=entity_names + relation_names,
    entity_names=entity_names,
    excluded_entity_names=[],
    relation_names=relation_names,
    excluded_relation_names=[],
    num_entities=2,
    frame_of_reference="intrinsic",
    w_range=(64, 64),
    h_range=(64, 64),
    theta_range=(0, 2 * math.pi),
    add_bbox=True,
    add_front=True,
    transform=None,
    canvas_size=(224, 224),
    num_samples=100,
)

In [ ]:
@interact(idx=(0, len(dataset) - 1))
def display_sample(idx=0):

    image_t, question_t, answer_t = dataset[idx]
    image = Image.fromarray(
        (255 * (dataset.std.view(-1, 1, 1) * image_t + dataset.mean.view(-1, 1, 1)))
        .permute(1, 2, 0)
        .numpy()
        .astype("uint8")
    )

    question = [dataset.idx2word[idx] for idx in question_t.tolist()]
    answer = bool(answer_t)
    display(image)
    print(question)
    print("Ground truth: ", answer)